# Projet 3 -  [Formatage des données propice à leur intégration dans une base de donnée relationnelle via le langage SQL](#M0)

# Table des matières

**[Création de la base de données appelée baseP3:](#M1)**

**[Question 15: Une table appelée population (formatage+ intégration + clé primaire)](#M2)**

**[Question 16: Une table appelée dispo_alim (formatage + intégration + clé primaire)](#M3)**

**[Question 17: Une table appelée equilibre_prod (formatage + intégration + clé primaire)](#M4)**

**[Question 18: Une table appelée sous_nutrition (formatage + intégration + clé primaire)](#M5)**


In [1]:
import pandas as pd
import numpy as np
import sqlite3, csv

## Création de la base de données appelée baseP3: <a name='M1'></a>

In [2]:
connexion = sqlite3.connect("baseP3.db")
curseur = connexion.cursor()

## Question 15: Une table appelée population (formatage+ intégration + clé primaire)<a name='M2'></a>

In [3]:
population = pd.read_csv('population_entre_2012_et_2017.csv')
population = population.drop(['Code Domaine','Domaine','Code Élément','Élément','Code Produit','Produit','Code année','Unité','Symbole','Description du Symbole','Note'],1)
population['Valeur'] *= 1000
population.columns = ["code_pays","pays",'année',"population"]
population["population"] = population["population"].apply(np.int64)
population.set_index('pays', drop =True, inplace = True)
population = population.drop(["Chine - RAS de Hong-Kong", "Chine - RAS de Macao", "Chine, Taiwan Province de", "Chine, continentale"])
population = population.reset_index()
population.head()

,pays,code_pays,année,population
0,Afghanistan,2,2012,30696958
1,Afghanistan,2,2013,31731688
2,Afghanistan,2,2014,32758020
3,Afghanistan,2,2015,33736494
4,Afghanistan,2,2016,34656032


In [4]:
population.to_csv(r'Export SQL\population.csv', index = False)

In [5]:
curseur.executescript('''
DROP TABLE IF EXISTS population;

CREATE TABLE IF NOT EXISTS population(
    pays       TEXT     NOT NULL,
    code_pays  INTEGER  NOT NULL,
    année      DATE (4) NOT NULL,
    population INTEGER  NOT NULL,
    PRIMARY KEY (
        code_pays,
        année
    )
);

DELETE FROM population;
''')
with open(r'Export SQL\population.csv','r',newline='', encoding='utf-8') as pop_export:
    pop = csv.DictReader(pop_export)
    donnees = [(i['pays'], i['code_pays'], i['année'], i['population']) for i in pop]
curseur.executemany('''INSERT INTO population VALUES(?,?,?,?);''', donnees)
connexion.commit()

## Question 16: Une table appelée dispo_alim (formatage + intégration + clé primaire)<a name='M3'></a>

In [12]:
ani = pd.read_csv('ani.csv')
veg = pd.read_csv('veg.csv')
ani["origin"] = "animal"
veg["origin"] = "végétal"
dispo_alim = ani.append(veg)
dispo_alim = dispo_alim.drop(['Code Domaine','Domaine','Code Élément','Code Année','Symbole','Description du Symbole'],1)
dispo_alim = dispo_alim.loc[dispo_alim['Élément'].isin(['Disponibilité alimentaire (Kcal/personne/jour)',
                                                        'Disponibilité alimentaire en quantité (kg/personne/an)',
                                                        'Disponibilité de matière grasse en quantité (g/personne/jour)',
                                                        'Disponibilité de protéines en quantité (g/personne/jour)'])]
dispo_alim = pd.pivot_table(dispo_alim, 'Valeur', ['Pays','Code Pays','Produit','Code Produit','origin','Année'],
                            'Élément').fillna(0)
dispo_alim['Disponibilité alimentaire en quantité (kg/personne/an)'] /= 1000
dispo_alim = dispo_alim.drop(["Chine - RAS de Hong-Kong", "Chine - RAS de Macao", "Chine, Taiwan Province de", 
                              "Chine, continentale"])
dispo_alim = dispo_alim.reset_index()
dispo_alim.columns = ['pays','code_pays','produit','code_produit','origin','année','dispo_alim_kcal_p_j','dispo_alim_tonnes',
                      'dispo_mat_gr','dispo_prot']
dispo_alim.head()

,pays,code_pays,produit,code_produit,origin,année,dispo_alim_kcal_p_j,dispo_alim_tonnes,dispo_mat_gr,dispo_prot
0,Afghanistan,2,Abats Comestible,2736,animal,2012,6.0,0.00182,0.22,0.81
1,Afghanistan,2,Abats Comestible,2736,animal,2013,5.0,0.00172,0.20,0.77
2,Afghanistan,2,"Agrumes, Autres",2614,végétal,2012,0.0,0.00062,0.00,0.01
3,Afghanistan,2,"Agrumes, Autres",2614,végétal,2013,1.0,0.00129,0.01,0.02
4,Afghanistan,2,Aliments pour enfants,2680,végétal,2012,1.0,0.00006,0.01,0.03


In [16]:
dispo_alim.to_csv(r'Export SQL\dispo_alim.csv', index = False)

In [17]:
curseur.executescript('''
DROP TABLE IF EXISTS dispo_alim;

CREATE TABLE IF NOT EXISTS dispo_alim(
    pays                TEXT     NOT NULL,
    code_pays           INTEGER  NOT NULL,
    produit             TEXT     NOT NULL,
    code_produit        INTEGER  NOT NULL,
    origin              TEXT     NOT NULL,
    année               DATE (4) NOT NULL,
    dispo_alim_kcal_p_j INTEGER,
    dispo_alim_tonnes   REAL,
    dispo_mat_gr        REAL,
    dispo_prot          REAL,
    PRIMARY KEY (
        code_pays,
        code_produit,
        année
    )
);

DELETE FROM dispo_alim;
''')
with open(r'Export SQL\dispo_alim.csv','r',newline='', encoding='utf-8') as d_a_export:
    d_a = csv.DictReader(d_a_export)
    donnees = [(i['pays'], i['code_pays'], i['produit'], i['code_produit'], i['origin'], i['année'], i['dispo_alim_kcal_p_j'], i['dispo_alim_tonnes'], i['dispo_mat_gr'], i['dispo_prot']) for i in d_a]
curseur.executemany('''INSERT INTO dispo_alim(pays, code_pays, produit, code_produit, origin, année, dispo_alim_kcal_p_j, dispo_alim_tonnes, dispo_mat_gr, dispo_prot) VALUES(?,?,?,?,?,?,?,?,?,?);''', donnees)
connexion.commit()

## Question 17: Une table appelée equilibre_prod (formatage + intégration + clé primaire)<a name='M4'></a>

In [18]:
ani = pd.read_csv('ani.csv')
veg = pd.read_csv('veg.csv')
equilibre_prod = ani.append(veg)
equilibre_prod = equilibre_prod.drop(['Code Domaine','Domaine','Code Élément','Code Année','Symbole','Description du Symbole'],1)
equilibre_prod = equilibre_prod.loc[equilibre_prod['Élément'].isin(['Aliments pour animaux','Autres Utilisations','Disponibilité intérieure','Nourriture','Pertes','Semences','Traitement'])]
equilibre_prod = pd.pivot_table(equilibre_prod, 'Valeur', ['Pays','Code Pays','Produit','Code Produit','Année'],'Élément').fillna(0).apply(np.int64)
equilibre_prod *= 1000
equilibre_prod = equilibre_prod.drop(["Chine - RAS de Hong-Kong", "Chine - RAS de Macao", "Chine, Taiwan Province de", "Chine, continentale"])
equilibre_prod = equilibre_prod.reset_index()
equilibre_prod.columns = ['pays','code_pays','produit','code_produit','année','alim_ani','autres_utilisations','dispo_int','nourriture','pertes','semences','transfo']
equilibre_prod.head()

,pays,code_pays,produit,code_produit,année,alim_ani,autres_utilisations,dispo_int,nourriture,pertes,semences,transfo
0,Afghanistan,2,Abats Comestible,2736,2012,0,0,54000,54000,0,0,0
1,Afghanistan,2,Abats Comestible,2736,2013,0,0,53000,53000,0,0,0
2,Afghanistan,2,"Agrumes, Autres",2614,2012,0,0,19000,18000,1000,0,0
3,Afghanistan,2,"Agrumes, Autres",2614,2013,0,0,41000,39000,2000,0,0
4,Afghanistan,2,Aliments pour enfants,2680,2012,0,0,2000,2000,0,0,0


In [19]:
equilibre_prod.to_csv(r'Export SQL\equilibre_prod.csv', index = False)

In [20]:
curseur.executescript('''
DROP TABLE IF EXISTS equilibre_prod;

CREATE TABLE IF NOT EXISTS equilibre_prod(
    pays                TEXT     NOT NULL,
    code_pays           INTEGER  NOT NULL,
    produit             TEXT     NOT NULL,
    code_produit        INTEGER  NOT NULL,
    année               DATE (4) NOT NULL,
    alim_ani            INTEGER,
    autres_utilisations INTEGER,
    dispo_int           INTEGER,
    nourriture          INTEGER,
    pertes              INTEGER,
    semences            INTEGER,
    transfo             INTEGER,
    PRIMARY KEY (
        code_pays,
        code_produit,
        année
    )
);

DELETE FROM equilibre_prod;
''')
with open(r'Export SQL\equilibre_prod.csv','r',newline='', encoding='utf-8') as e_p_export:
    e_p = csv.DictReader(e_p_export)
    donnees = [(i['pays'], i['code_pays'], i['produit'], i['code_produit'], i['année'], i['alim_ani'], i['autres_utilisations'], i['dispo_int'], i['nourriture'], i['pertes'], i['semences'], i['transfo']) for i in e_p]
curseur.executemany('''INSERT INTO equilibre_prod VALUES(?,?,?,?,?,?,?,?,?,?,?,?);''', donnees)
connexion.commit()

## Question 18: Une table appelée sous_nutrition (formatage + intégration + clé primaire)<a name='M5'></a>

In [21]:
sous_nutrition = pd.read_csv('sous_alimentes.csv').fillna(0)
sous_nutrition = pd.pivot_table(sous_nutrition, 'Valeur', ['Zone','Code zone','Année'], aggfunc = sum)
for i in sous_nutrition.columns: sous_nutrition[i] = pd.to_numeric(sous_nutrition[i], errors='coerce') # pour convertir des '<0.1' en Nan
sous_nutrition = sous_nutrition.fillna(0.1) # ordre de grandeur de base avant conversion
sous_nutrition *= 1000000
sous_nutrition.rename(index ={'2011-2013':'2012','2012-2014':'2013','2013-2015':'2014','2014-2016':'2015','2015-2017':'2016','2016-2018':'2017'}, inplace = True)
sous_nutrition = sous_nutrition.drop(["Chine - RAS de Hong-Kong", "Chine - RAS de Macao", "Chine, Taiwan Province de", "Chine, continentale"]).apply(np.int64)
sous_nutrition = sous_nutrition.reset_index()
sous_nutrition.columns = ['pays','code_pays','année','nb_personnes'] 
sous_nutrition.head()

,pays,code_pays,année,nb_personnes
0,Afghanistan,2,2012,7100000
1,Afghanistan,2,2013,7900000
2,Afghanistan,2,2014,8800000
3,Afghanistan,2,2015,9600000
4,Afghanistan,2,2016,10200000


In [22]:
sous_nutrition.to_csv(r'Export SQL\sous_nutrition.csv', index = False)

In [23]:
curseur.executescript('''
DROP TABLE IF EXISTS sous_nutrition;

CREATE TABLE IF NOT EXISTS sous_nutrition(
    pays         TEXT     NOT NULL,
    code_pays    INTEGER  NOT NULL,
    année        DATE (4) NOT NULL,
    nb_personnes INTEGER  DEFAULT (0),
    PRIMARY KEY (
        code_pays,
        année
    )
);

DELETE FROM sous_nutrition;
''')
with open(r'Export SQL\sous_nutrition.csv','r',newline='', encoding='utf-8') as s_n_export:
    s_n = csv.DictReader(s_n_export)
    donnees = [(i['pays'], i['code_pays'], i['année'], i['nb_personnes']) for i in s_n]
curseur.executemany('''INSERT INTO sous_nutrition VALUES(?,?,?,?);''', donnees)
connexion.commit()

In [24]:
connexion.close()